# 5 서포트 벡터 머신 (support vector machine - SVM)

다목적 머신러닝 모델

복잡한 분류 문제, 작거나 중간 크기의 데이터셋

## 5.1 선형 SVM 분류

SVM의 기본 아이디어는 그림으로 설명하는 것이 가장 좋다.

SVM 분류기의 결정 경계는 두 개의 클래스를 나눌 뿐만 아니라 제일 가까운 훈련 샘플로부터 가능한 한 멀리 떨어져 있다.

클래스 사이에 가장 폭이 넓은 도로를 찾는 것으로 생각할 수 있다. (라지 마진 분류라고도 함.)

도로 바깥쪽에 훈련 샘플을 더 추가해도 결정 경계에는 전혀 영향을 및피지 않는다.

도로 경계에 위치한 샘플에 의해 전적으로 결정(또는 의지)된다.

이런 샘플을 서포트 벡터라고 한다.

### 5.1.1 소프트 마진 분류

모든 샘플들이 도로 바깥쪽에 올바르게 분류되어 있다면 이를 하드 마진 분류라고 한다.

하드 마진 분류에는 2가지 문제점이 있다.

1. 데이터가 선형적으로 구분될 수 있어야 제대로 작동한다.
2. 이상치에 민감하다.

이런 문제를 피하려면 좀 더 유연한 모델이 필요하다.

도로의 폭을 가능한 넓게 유지하는 것과, 마진 오류 사이에 적절한 균형을 잡아야 한다.

이를 소프트 마진 분류라고 한다.

#### 하이퍼파라미터 C (마진 오류)

사이킷런의 SVM 모델을 만들 때 여러 하이퍼파라미터를 지정할 수 있다.

C는 그 중에 하나이다.

이를 낮게 설정하면(C=1) 넓은 마진 오류를 가지며 유연해진다. (규제)

이를 높게 설정하면(C=100) 적은 마진 오류를 가지며 하드 마진 분류에 가깝다.

마진 오류는 일반적으로 적은 것이 좋은데, 마진 오류가 많지만 일반화가 더 잘 되는 모델을 택할 수도 있다.

In [9]:
'''
붓꽃 데이터셋을 적재하고, 특성 스케일을 변경하고, Iris-Virginia 품종을 감지하기 위해 선형 SVM 모델을 훈련시켜보자.
(C=1과 힌지 손실 함수를 적용한 LinearSVC 클래스를 사용한다.)
'''

import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X, y)

print(svm_clf.predict([[5.5, 1.7]]))

[1.]


## 5.2 비선형 SVM 분류

선형 SVM 분류기가 효율적이고 많은 경우에 아주 잘 동작하지만, 선형적으로 분류할 수 없는 데이터셋이 많다.

비선형 데이터셋을 다루는 한 가지 방법은 다항 특성과 같은 특성을 더 추가하는 것이다.

이렇게 하면 선형적으로 구분되는 데이터셋이 만들어질 수 있다.

사이킷런을 사용하여 이를 구현하려면 PolynomialFeatures 변환기와 StandardScaler, LinearSVC를 연결하여 Pipeline을 만든다.

In [16]:
'''
이를 moons 데이터셋에 적용해보자.
이 데이터셋은 마주보는 두 개의 반원 모양으로 데이터 포인트가 놓여 있는 이진 분류를 위한 작은 데이터셋이다.
'''

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

### 5.2.1 다항식 커널

다항식 특성을 추가하는 것은 간단하고 모든 머신러닝 알고리즘에서 잘 작동한다.

하지만 낮은 차수의 다항식은 매우 복잡한 데이터셋을 잘 표현하지 못하고 높은 차수의 다항식은 굉장히 많은 특성을 추가하므로 모델을 느리게 만든다.

다행히도 SVM을 사용할 땐 커널 트릭이라는 거의 기적에 가까운 수학적 기교를 적용할 수 있다.

커널 트릭은 실제로는 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과를 얻을 수 있다.

In [19]:
'''
이 기법은 SVC 파이썬 클래스에 구현되어 있다.
'''

from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

'''
결과는 [그림 5-7] 왼쪽이다.
오른쪽 그래프는 10차 다항식 커널을 사용한 또 다른 SVM 분류기이다.
모델이 과대적합이라면 다항식의 차수를 줄여야하고, 반대로 과소적합이라면 차수를 늘려야 한다.
매개변수 coef0는 모델이 높은 차수와 낮은 차수에 얼마나 영향을 받을지 조절한다.
'''
pass

### 5.2.2 유사도 특성
